# Part 1: Determine what families to use for simulation. We want then to have roughly the same size, such that effective sample size and number of families are closely related.

In [1]:
from src.benchmarking.simulated_data_benchmarking import get_family_sizes, get_families_within_cutoff, fig_family_sizes
from src.caching import set_cache_dir, set_use_hash
from src.utils import get_amino_acids
from src.phylogeny_estimation import fast_tree
from src.benchmarking.simulated_data_benchmarking import create_trivial_contact_maps, get_families, subset_msa_to_leaf_nodes
from src.simulation import simulate_msas
from src.counting import count_transitions
from src.estimation import jtt_ipw, quantized_transitions_mle

MSA_DIR = "/export/home/users/sprillo/Git/Phylo-correction/cs267_data/msas_1024_seqs_None_sites_LG_FastTree.txt-d15ceeb4_RM_20_cats/"
LG_PATH = "./data/rate_matrices/lg.txt"
LG_STATIONARY_PATH = "./data/rate_matrices/lg_stationary.txt"
LG_X_LG_PATH = "./data/rate_matrices/lg_x_lg.txt"
LG_X_LG_STATIONARY_PATH = "./data/rate_matrices/lg_x_lg_stationary.txt"
EQU_PATH = "./data/rate_matrices/equ.txt"

set_cache_dir("_cache")

[2022-04-30 00:04:47,302] - caching - INFO - Setting cache directory to: _cache


In [2]:
# fig_family_sizes(
#     msa_dir=MSA_DIR,
# )

In [3]:
families = get_families_within_cutoff(
    msa_dir=MSA_DIR,
    min_num_sites=195,
    max_num_sites=225,
    min_num_sequences=1024,
    max_num_sequences=1024,
)
families = sorted(families)[:32]

In [4]:
fast_tree_output_dirs = fast_tree(
    msa_dir=MSA_DIR,
    families=families,
    rate_matrix_path=LG_PATH,
    num_rate_categories=20,
    num_processes=32,
)

In [5]:
trivial_contact_map_dir = create_trivial_contact_maps(
    msa_dir=MSA_DIR,
    families=families,
    states=get_amino_acids(),
)["output_contact_map_dir"]

In [6]:
simulated_msa_dir = simulate_msas(
    tree_dir=fast_tree_output_dirs['output_tree_dir'],
    site_rates_dir=fast_tree_output_dirs['output_site_rates_dir'],
    contact_map_dir=trivial_contact_map_dir,
    families=families,
    amino_acids=get_amino_acids(),
    pi_1_path=LG_STATIONARY_PATH,
    Q_1_path=LG_PATH,
    pi_2_path=LG_X_LG_STATIONARY_PATH,
    Q_2_path=LG_X_LG_PATH,
    strategy="all_transitions",
    random_seed=0,
    num_processes=32,
    use_cpp_implementation=False,
)["output_msa_dir"]

In [7]:
quantization_points = [("%.5f" % (0.06 * 1.1 ** i)) for i in range(-50, 51, 1)]

count_matrices__edge_gt__dir = count_transitions(
    tree_dir=fast_tree_output_dirs['output_tree_dir'],
    msa_dir=simulated_msa_dir,
    site_rates_dir=fast_tree_output_dirs['output_site_rates_dir'],
    families=families,
    amino_acids=get_amino_acids(),
    quantization_points=quantization_points,
    edge_or_cherry="edge",
    num_processes=32,
    use_cpp_implementation=False,
)["output_count_matrices_dir"]

In [8]:
jtt_ipw__edge_gt__dir = jtt_ipw(
    count_matrices_path=os.path.join(count_matrices__edge_gt__dir, "result.txt"),
    mask_path=None,
    use_ipw=True,
    normalize=False,
)["output_rate_matrix_dir"]

In [9]:
rate_matrix__edge_gt__dir = quantized_transitions_mle(
    count_matrices_path=os.path.join(count_matrices__edge_gt__dir, "result.txt"),
    initialization_path=os.path.join(jtt_ipw__edge_gt__dir, "result.txt"),
    mask_path=None,
    stationary_distribution_path=None,
    rate_matrix_parameterization="pande_reversible",
    device="cpu",
    learning_rate=1e-1,
    num_epochs=200,
    do_adam=True,
)["output_rate_matrix_dir"]
print(rate_matrix__edge_gt__dir)

_cache/quantized_transitions_mle/88fd9472dc6fcb4a7e534743686cbe52605c45855dce0265c23b0d69e44bc353a3258b912a2214c343dad5cc7239309ddd4651abc112c5c629c3ed73c9f5f728/output_rate_matrix_dir


In [10]:
count_matrices__cherry_gt__dir = count_transitions(
    tree_dir=fast_tree_output_dirs['output_tree_dir'],
    msa_dir=simulated_msa_dir,
    site_rates_dir=fast_tree_output_dirs['output_site_rates_dir'],
    families=families,
    amino_acids=get_amino_acids(),
    quantization_points=quantization_points,
    edge_or_cherry="cherry",
    num_processes=32,
    use_cpp_implementation=False,
)["output_count_matrices_dir"]

In [11]:
jtt_ipw__cherry_gt__dir = jtt_ipw(
    count_matrices_path=os.path.join(count_matrices__cherry_gt__dir, "result.txt"),
    mask_path=None,
    use_ipw=True,
    normalize=False,
)["output_rate_matrix_dir"]

In [12]:
rate_matrix__cherry_gt__dir = quantized_transitions_mle(
    count_matrices_path=os.path.join(count_matrices__cherry_gt__dir, "result.txt"),
    initialization_path=os.path.join(jtt_ipw__cherry_gt__dir, "result.txt"),
    mask_path=None,
    stationary_distribution_path=None,
    rate_matrix_parameterization="pande_reversible",
    device="cpu",
    learning_rate=1e-1,
    num_epochs=200,
    do_adam=True,
)["output_rate_matrix_dir"]
print(rate_matrix__cherry_gt__dir)

_cache/quantized_transitions_mle/6e411016a15f9e5c38dd73eb69ea1cdb8ddf57e3cbe479c6f2d68a4e76cccfef61d131a4da2b98fae77bdd3162d10e114d376e0fa9aeada5e9dd30caf18bb069/output_rate_matrix_dir


In [16]:
simulated_msa_leaves_dir = subset_msa_to_leaf_nodes(
    msa_dir=simulated_msa_dir,
    families=families,
    states=get_amino_acids(),
)["output_msa_dir"]
print(simulated_msa_leaves_dir)

_cache/subset_msa_to_leaf_nodes/6891acc81ed49a07cee1ebeca7ae2d8ae1999b07c17b8b0d6ee7ac4f4b43fb6cb32805203b90942f60931ebe75bf18f2f6095c8458aa0a0f491efc9d384b4e2d/output_msa_dir


In [15]:
fast_tree_output__simulation_iter_1__dirs = fast_tree(
    msa_dir=simulated_msa_leaves_dir,
    families=families,
    rate_matrix_path=EQU_PATH,
    num_rate_categories=20,
    num_processes=32,
)

  0%|          | 0/32 [00:00<?, ?it/s]


TypeError: expected str, bytes or os.PathLike object, not dict

In [ ]:
count_matrices__cherry_iter_1__dir = count_transitions(
    tree_dir=fast_tree_output__simulation_iter_1__dirs['output_tree_dir'],
    msa_dir=simulated_msa_leaves_dir,
    site_rates_dir=fast_tree_output__simulation_iter_1__dirs['output_site_rates_dir'],
    families=families,
    amino_acids=get_amino_acids(),
    quantization_points=quantization_points,
    edge_or_cherry="cherry",
    num_processes=32,
    use_cpp_implementation=False,
)["output_count_matrices_dir"]
print(count_matrices__cherry_iter_1__dir)


In [ ]:
jtt_ipw__cherry_iter_1__dir = jtt_ipw(
    count_matrices_path=os.path.join(count_matrices__cherry_iter_1__dir, "result.txt"),
    mask_path=None,
    use_ipw=True,
    normalize=False,
)["output_rate_matrix_dir"]
print(jtt_ipw__cherry_iter_1__dir)

In [ ]:
rate_matrix__cherry_iter_1__dir = quantized_transitions_mle(
    count_matrices_path=os.path.join(count_matrices__cherry_iter_1__dir, "result.txt"),
    initialization_path=os.path.join(jtt_ipw__cherry_iter_1__dir, "result.txt"),
    mask_path=None,
    stationary_distribution_path=None,
    rate_matrix_parameterization="pande_reversible",
    device="cpu",
    learning_rate=1e-1,
    num_epochs=200,
    do_adam=True,
)["output_rate_matrix_dir"]
print(rate_matrix__cherry_iter_1__dir)

In [ ]:
fast_tree_output__simulation_iter_2__dirs = fast_tree(
    msa_dir=simulated_msa_leaves_dir,
    families=families,
    rate_matrix_path=os.path.join(rate_matrix__cherry_iter_1__dir, "result.txt"),
    num_rate_categories=20,
    num_processes=32,
)

In [ ]:
count_matrices__cherry_iter_2__dir = count_transitions(
    tree_dir=fast_tree_output__simulation_iter_2__dirs['output_tree_dir'],
    msa_dir=simulated_msa_dir,
    site_rates_dir=fast_tree_output__simulation_iter_2__dirs['output_site_rates_dir'],
    families=families,
    amino_acids=get_amino_acids(),
    quantization_points=quantization_points,
    edge_or_cherry="cherry",
    num_processes=32,
    use_cpp_implementation=False,
)["output_count_matrices_dir"]
print(count_matrices__cherry_iter_2__dir)


In [ ]:
jtt_ipw__cherry_iter_2__dir = jtt_ipw(
    count_matrices_path=os.path.join(count_matrices__cherry_iter_2__dir, "result.txt"),
    mask_path=None,
    use_ipw=True,
    normalize=False,
)["output_rate_matrix_dir"]
print(jtt_ipw__cherry_iter_2__dir)

In [ ]:
rate_matrix__cherry_iter_2__dir = quantized_transitions_mle(
    count_matrices_path=os.path.join(count_matrices__cherry_iter_2__dir, "result.txt"),
    initialization_path=os.path.join(jtt_ipw__cherry_iter_2__dir, "result.txt"),
    mask_path=None,
    stationary_distribution_path=None,
    rate_matrix_parameterization="pande_reversible",
    device="cpu",
    learning_rate=1e-1,
    num_epochs=200,
    do_adam=True,
)["output_rate_matrix_dir"]
print(rate_matrix__cherry_iter_2__dir)

In [ ]:
fast_tree_output__simulation_iter_3__dirs = fast_tree(
    msa_dir=simulated_msa_dir,
    families=families,
    rate_matrix_path=os.path.join(rate_matrix__cherry_iter_2__dir, "result.txt"),
    num_rate_categories=20,
    num_processes=32,
)

In [ ]:
count_matrices__cherry_iter_3__dir = count_transitions(
    tree_dir=fast_tree_output__simulation_iter_3__dirs['output_tree_dir'],
    msa_dir=simulated_msa_dir,
    site_rates_dir=fast_tree_output__simulation_iter_3__dirs['output_site_rates_dir'],
    families=families,
    amino_acids=get_amino_acids(),
    quantization_points=quantization_points,
    edge_or_cherry="cherry",
    num_processes=32,
    use_cpp_implementation=False,
)["output_count_matrices_dir"]
print(count_matrices__cherry_iter_3__dir)


In [ ]:
jtt_ipw__cherry_iter_3__dir = jtt_ipw(
    count_matrices_path=os.path.join(count_matrices__cherry_iter_3__dir, "result.txt"),
    mask_path=None,
    use_ipw=True,
    normalize=False,
)["output_rate_matrix_dir"]
print(jtt_ipw__cherry_iter_3__dir)

In [ ]:
rate_matrix__cherry_iter_3__dir = quantized_transitions_mle(
    count_matrices_path=os.path.join(count_matrices__cherry_iter_3__dir, "result.txt"),
    initialization_path=os.path.join(jtt_ipw__cherry_iter_3__dir, "result.txt"),
    mask_path=None,
    stationary_distribution_path=None,
    rate_matrix_parameterization="pande_reversible",
    device="cpu",
    learning_rate=1e-1,
    num_epochs=200,
    do_adam=True,
)["output_rate_matrix_dir"]
print(rate_matrix__cherry_iter_3__dir)

In [ ]:
fast_tree_output__simulation_iter_oracle__dirs = fast_tree(
    msa_dir=simulated_msa_dir,
    families=families,
    rate_matrix_path=LG_PATH,
    num_rate_categories=20,
    num_processes=32,
)

In [ ]:
count_matrices__cherry_iter_oracle__dir = count_transitions(
    tree_dir=fast_tree_output__simulation_iter_oracle__dirs['output_tree_dir'],
    msa_dir=simulated_msa_dir,
    site_rates_dir=fast_tree_output__simulation_iter_oracle__dirs['output_site_rates_dir'],
    families=families,
    amino_acids=get_amino_acids(),
    quantization_points=quantization_points,
    edge_or_cherry="cherry",
    num_processes=32,
    use_cpp_implementation=False,
)["output_count_matrices_dir"]
print(count_matrices__cherry_iter_oracle__dir)


In [ ]:
jtt_ipw__cherry_iter_oracle__dir = jtt_ipw(
    count_matrices_path=os.path.join(count_matrices__cherry_iter_oracle__dir, "result.txt"),
    mask_path=None,
    use_ipw=True,
    normalize=False,
)["output_rate_matrix_dir"]
print(jtt_ipw__cherry_iter_oracle__dir)

In [ ]:
rate_matrix__cherry_iter_oracle__dir = quantized_transitions_mle(
    count_matrices_path=os.path.join(count_matrices__cherry_iter_oracle__dir, "result.txt"),
    initialization_path=os.path.join(jtt_ipw__cherry_iter_oracle__dir, "result.txt"),
    mask_path=None,
    stationary_distribution_path=None,
    rate_matrix_parameterization="pande_reversible",
    device="cpu",
    learning_rate=1e-1,
    num_epochs=200,
    do_adam=True,
)["output_rate_matrix_dir"]
print(rate_matrix__cherry_iter_oracle__dir)